In [ ]:
!pip install google.cloud.translate

In [ ]:
import os
# public
import pandas as pd
from tqdm import tqdm
from google.cloud import translate_v2 as translate

In [ ]:
# a method that calls the cloud translator
def translate_text(source: str, target: str, text: str) -> dict:
    """
    Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(
        text
        , source_language=source
        , target_language=target
    )
    return result

def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [ ]:
src_lan = 'en'
# target language
tgt_lan = 'es'
language = "Spanish"


RESOURCE = ''
DATA =  RESOURCE+'/data/'+language
GOLD_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TRANS_FILE = os.path.join(DATA, 'trans-sentences-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'data'

KEY = os.path.join(RESOURCE, 'key')
# RAW_SENSE_TSV = os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
# RAW_TOKENS_TSV =  os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'SemEval2025_EAMT_Samples.tsv')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'leafy-winter-436121-s1-c359b02fd0ef.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS

# DATA

In [ ]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t')
sentence_df.head()
sents = sentence_df["source"].tolist()
print(sentence_df["target"].tolist())
targets = sentence_df["target_locale"].tolist()
sources = sentence_df["source_locale"].tolist()
langs = [ [sources[x],targets[x]] for x in range(len(sources))]
print(langs)
# sents

In [ ]:
sids = sentence_df["id"].tolist()
# sids = [i.replace('d', 'i') for i in sids]
sids[:4]

In [ ]:
len(sids), len(sents)

# TRANSLATE

In [ ]:
# get languages
client = translate.Client()
languages = client.get_languages()
languages

In [ ]:
src_lan, tgt_lan

In [ ]:
# sample test
sentence = sents[0]
test = translate_text(langs[0][0], langs[0][1], sentence)
test

In [ ]:
# translating all the sentences from English to target language
tgt_sents = []
for s in tqdm(range(len(sents))):
    tgt_s = translate_text(langs[s][0], langs[s][1], sents[s])
    tgt_sents.append(tgt_s)

In [ ]:
print(tgt_sents)
tgt_sents = [s['translatedText'] for s in tgt_sents]

In [ ]:
# Save to TSV 
sentence_df["Cloud Translation"] = tgt_sents
sentence_df.head()

In [ ]:
sentence_df.to_csv(RAW_SENTENCE_TSV,sep='\t',index=False)